In [2]:
from youtubesearchpython import VideosSearch,PlaylistsSearch
from langchain_groq import ChatGroq
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from dotenv import load_dotenv
load_dotenv()

/home/rohan.dubey@simform.dom/.pyenv/versions/3.9.22/envs/agent-project/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
model1=ChatGoogleGenerativeAI(model='gemini-2.5-pro')

model2=ChatGroq(model='llama-3.1-8b-instant')

In [8]:
query ="LanGraph Tutorials"

In [7]:
from youtube_search import YoutubeSearch
from langchain_core.runnables import RunnableLambda

In [94]:
run=RunnableLambda(lambda x : YoutubeSearch(x).to_json())

In [95]:
run.invoke("LangGraph")

'{"videos": [{"id": "gqvFmK7LpDo", "thumbnails": ["https://i.ytimg.com/vi/gqvFmK7LpDo/hq720.jpg?sqp=-oaymwEjCOgCEMoBSFryq4qpAxUIARUAAAAAGAElAADIQj0AgKJDeAE=&rs=AOn4CLAcQ4j9ldrf9uhWx03RCmnhFoW73Q", "https://i.ytimg.com/vi/gqvFmK7LpDo/hq720.jpg?sqp=-oaymwEXCNAFEJQDSFryq4qpAwkIARUAAIhCGAE=&rs=AOn4CLBbkq22oDmASFrRvuyDRzYQP8aV0w"], "title": "Tutorial 1-Getting Started With LangGraph- Building Stateful Multi AI Agents", "long_desc": null, "channel": "Krish Naik", "duration": "40:23", "views": "100,192 views", "publish_time": "8 months ago", "url_suffix": "/watch?v=gqvFmK7LpDo&pp=ygUJTGFuZ0dyYXBo"}, {"id": "qAF1NjEVHhY", "thumbnails": ["https://i.ytimg.com/vi/qAF1NjEVHhY/hq720.jpg?sqp=-oaymwEjCOgCEMoBSFryq4qpAxUIARUAAAAAGAElAADIQj0AgKJDeAE=&rs=AOn4CLCMF4eEECgQUETQq7YITCMk585qcg", "https://i.ytimg.com/vi/qAF1NjEVHhY/hq720.jpg?sqp=-oaymwEXCNAFEJQDSFryq4qpAwkIARUAAIhCGAE=&rs=AOn4CLBMWD1Zfoqu0TJhQXaLEcYVGj_0MQ"], "title": "LangChain vs LangGraph: A Tale of Two Frameworks", "long_desc": null, "cha

In [41]:
result=YoutubeSearch(query,max_results=3).to_dict()

In [52]:
print(result[1])

{'id': 'aHCDrAbH_go', 'thumbnails': ['https://i.ytimg.com/vi/aHCDrAbH_go/hq720.jpg?sqp=-oaymwEjCOgCEMoBSFryq4qpAxUIARUAAAAAGAElAADIQj0AgKJDeAE=&rs=AOn4CLCZLUcD4ToBFaIEpZJYFu4s4qsRqw', 'https://i.ytimg.com/vi/aHCDrAbH_go/hq720.jpg?sqp=-oaymwEXCNAFEJQDSFryq4qpAwkIARUAAIhCGAE=&rs=AOn4CLDSfKErX0Wju8KeXJjKM00XkgpMzQ'], 'title': 'Building Effective Agents with LangGraph', 'long_desc': None, 'channel': 'LangChain', 'duration': '31:50', 'views': '96,495 views', 'publish_time': '3 months ago', 'url_suffix': '/watch?v=aHCDrAbH_go&pp=ygUSTGFuR3JhcGggVHV0b3JpYWxz0gcJCYsJAYcqIYzv'}


In [54]:
for a in result[1]:
    print(f"{a}  {result[2][a]}")

id  PqS1kib7RTw
thumbnails  ['https://i.ytimg.com/vi/PqS1kib7RTw/hq720.jpg?sqp=-oaymwEjCOgCEMoBSFryq4qpAxUIARUAAAAAGAElAADIQj0AgKJDeAE=&rs=AOn4CLCB8M0TncSFwM0c-SSpBYwsAKwoxA', 'https://i.ytimg.com/vi/PqS1kib7RTw/hq720.jpg?sqp=-oaymwEXCNAFEJQDSFryq4qpAwkIARUAAIhCGAE=&rs=AOn4CLAzxtxSNt_snIe6071ofqLJ2Y_QDQ']
title  LangGraph Crash Course with code examples
long_desc  None
channel  Sam Witteveen
duration  39:01
views  111,345 views
publish_time  1 year ago
url_suffix  /watch?v=PqS1kib7RTw&pp=ygUSTGFuR3JhcGggVHV0b3JpYWxz


In [4]:
from langchain.tools import YouTubeSearchTool
from langchain.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage, SystemMessage
from langchain_core.tools import tool
from langchain_core.output_parsers import StrOutputParser
import os

In [8]:
@tool   
def find_youtube_resources(topic:str)->str:
    """
    Find YouTube playlists/videos for a given topic
    Returns list of resources with details
    """
    # Initialize YouTube search tool
    yt_search = ()
    
    # Search YouTube
    raw_results = YoutubeSearch(f"{topic} course OR tutorial OR playlist").to_json()
    
    # Process results with LLM to extract structured information
    prompt = ChatPromptTemplate([
        ("system",
            """You are a YouTube resource curator. Extract the following details from YouTube search results:\n"
            "1. Title\n"
            "2. Channel Name\n"
            "3. URL\n"
            "4. Duration (if available)\n"
            "5. Brief description\n"
            "Format as a numbered list with clear labels for each field.\n"
            "Include only the most relevant 5 results."""),
        ("human","Search results : {results}")
    ])
    chain=RunnableLambda(lambda x : YoutubeSearch(x).to_json())  |prompt | model2 | StrOutputParser()
    formatted_results = chain.invoke(topic)
    #formatted_results = model2.invoke(prompt.format_messages(input=topic))
    
    return formatted_results

In [9]:
result=find_youtube_resources.invoke("LangChain")

In [10]:
print(result)

Here are the most relevant 5 YouTube search results for "LangChain" along with their details:

1. **Title**: LangChain explained - The hottest new Python framework
**Channel Name**: AssemblyAI
**URL**: https://www.youtube.com/watch?v=RoR4XJw8wIc
**Duration**: 3:03
**Brief description**: A video explaining LangChain, a new Python framework for building generative AI models.

2. **Title**: LangChain Crash Course For Beginners | LangChain Tutorial
**Channel Name**: codebasics
**URL**: https://www.youtube.com/watch?v=nAmC7SoVLd8
**Duration**: 46:07
**Brief description**: A comprehensive tutorial for beginners on LangChain, covering its basics and concepts.

3. **Title**: Complete Langchain Course For Generative AI In 3 Hours
**Channel Name**: Krish Naik
**URL**: https://www.youtube.com/watch?v=swCPic00c30
**Duration**: 3:13:35
**Brief description**: A 3-hour course on LangChain for building generative AI models.

4. **Title**: LangChain Master Class For Beginners 2024 [+20 Examples, LangCh